In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv("data/train.csv")
train["date"] = pd.to_datetime(train["date"], format = "%Y%m%d")
test = pd.read_csv("data/test.csv")

price = (
    train
    .assign(mode_price_202204 = lambda df: df["mode_price"].mask(~df["date"].between("2022-04-17", "2022-04-30"), np.nan))
    .assign(mode_price_202105 = lambda df: df["mode_price"].mask(~df["date"].between("2021-05-01", "2021-05-31"), np.nan))
    .assign(mode_price_202005 = lambda df: df["mode_price"].mask(~df["date"].between("2020-05-01", "2020-05-31"), np.nan))
    .groupby(["kind"], as_index = False)[["mode_price_202204", "mode_price_202105", "mode_price_202005"]].mean()
    .assign(mode_price = lambda df: (
        df["mode_price_202204"]
        .mask(df["kind"].isin(["きゅうり", "そらまめ", "トマト", "ピーマン", "ミニトマト", "レタス"]), df["mode_price_202105"])
        .mask(df["kind"].isin(["だいこん", "ねぎ", "セルリー"]), df["mode_price_202005"])
    ))
)

submit = test[["kind", "date"]].merge(price[["kind", "mode_price"]], how = "left", on = "kind")
submit.to_csv("submit/submit_20220807_04.csv", header = True, index = False)